In [27]:
# sat xarray to np
import xarray as xr
from ocf_blosc2 import Blosc2
import numpy as np
from datetime import datetime
import os
from tqdm import tqdm
def process(year, month):
    import xarray as xr
    from ocf_blosc2 import Blosc2
    import numpy as np
    from datetime import datetime
    import os
    from tqdm import tqdm
    data = xr.open_dataset(
        f"/data/satellite-nonhrv/{year}/{month}.zarr.zip", engine="zarr", chunks="auto"
    )
    for time in tqdm(data["time"].to_numpy()):
        dt = to_datetime(time)
        folder_name = base_folder + dt.strftime('%y-%m-%d') + "/"
        file_name = dt.strftime('%H:%M:%S') + ".npz"
        total_name = folder_name + file_name
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        data_np = data["data"].sel(time=time).to_numpy()
        np.savez(total_name, data_np) # save
def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)
    
base_folder = "/data/sat_np/"
from joblib import Parallel, delayed
parallel = Parallel(n_jobs=4, return_as="generator")
for year in range(2021, 2022):
    Parallel(n_jobs=6)(delayed(process)(year, month) for month in range(1, 13))

100%|██████████| 6727/6727 [09:26<00:00, 11.87it/s]


In [30]:
# weather xarray to np
import xarray as xr
from ocf_blosc2 import Blosc2
import numpy as np
from datetime import datetime
import os
from tqdm import tqdm

base_folder = "/data/weather_np/"

def process(year, month):
    import xarray as xr
    from ocf_blosc2 import Blosc2
    import numpy as np
    from datetime import datetime
    import os
    from tqdm import tqdm
    data = xr.open_dataset(
        f"/data/weather/{year}/{month}.zarr.zip", engine="zarr", chunks="auto"
    )
    NWP_FEATURES = [
    "t_500", "clcl", "alb_rad", "tot_prec", "ww",
    "relhum_2m", "h_snow", "aswdir_s", "td_2m", "omega_1000"
    ]
    data = data[NWP_FEATURES]
    for time in tqdm(data["time"].to_numpy()):
        dt = to_datetime(time)
        folder_name = base_folder + dt.strftime('%y-%m-%d') + "/"
        file_name = dt.strftime('%H:%M:%S') + ".npz"
        total_name = folder_name + file_name
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        datas_np = []
        for var in NWP_FEATURES:
            data_np = data[var].sel(time=time).to_numpy()
            datas_np.append(data_np)
        datas_np = np.array(datas_np)
        np.savez(total_name, datas_np) # save
def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)
    
from joblib import Parallel, delayed
for year in range(2021, 2022):
    Parallel(n_jobs=6)(delayed(process)(year, month) for month in range(1, 13))

(10, 305, 289)


  0%|          | 0/524 [00:00<?, ?it/s].96it/s]

(10, 305, 289)


  0%|          | 0/570 [00:00<?, ?it/s].39it/s]

(10, 305, 289)
(10, 305, 289)
(10, 305, 289)
(10, 305, 289)


100%|██████████| 589/589 [03:19<00:00,  2.95it/s]
